## Establishing Connection

In [2]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
user = "neo4j"
password = "12345678"  
database = "moviesdb"

driver = GraphDatabase.driver(uri, auth=(user, password))

def test_connection():
    with driver.session(database=database) as session:
        result = session.run("RETURN 'Connected to Neo4j!' AS msg")
        for record in result:
            print(record["msg"])

test_connection()

Connected to Neo4j!


## Creating Nodes


In [4]:

# Creating nodes

def run_query(query, params=None):
    with driver.session(database="metadatatagging") as session:
        result = session.run(query, params or {})
        return [record.data() for record in result]
    
def create_movie(tx, imdb_id=-1, title=None, year=-1, 
                 akas=None, metascore=None, imdb_rating=None, imdb_votes=None, 
                 budget=None, countries=None, age_restrictions=None, 
                 keywords=None, taglines=None, plot=None):

    query = """
    MERGE (m:Movie {imdb_id: $imdb_id})
    SET m.title = $title,
        m.year = $year,
        m.akas = $akas,
        m.metascore = $metascore,
        m.imdb_rating = $imdb_rating,
        m.imdb_votes = $imdb_votes,
        m.budget = $budget,
        m.countries = $countries,
        m.age_restrictions = $age_restrictions,
        m.keywords = $keywords,
        m.taglines = $taglines,
        m.plot = $plot
    """
    tx.run(query,
           imdb_id=imdb_id, title=title, year=year, akas=akas,
           metascore=metascore, imdb_rating=imdb_rating, imdb_votes=imdb_votes,
           budget=budget, countries=countries,
           age_restrictions=age_restrictions,
           keywords=keywords, taglines=taglines,
           plot=plot)
    


def create_genre(tx, name):
    tx.run("MERGE (g:Genre {name: $name})", name=name)

def create_person(tx, name, role=None):
    query = """
    MERGE (p:Person {name: $name})
    SET p.roles = CASE 
        WHEN p.roles IS NULL THEN [$role]
        WHEN NOT $role IN p.roles THEN p.roles + $role
        ELSE p.roles
    END
    """
    tx.run(query, name=name, role=role)


def create_company(tx, name):
    tx.run("MERGE (c:Company {name: $name})", name=name)

def create_award(tx, name):
    tx.run("MERGE (a:Award {name: $name})", name=name)
    
def create_review(tx, movie_id, review_id, text, reviewer, score=None, sentiment=None, sentiment_score=None):
    query = """
    MERGE (m:Movie {imdb_id: $movie_id})
      ON CREATE SET m.title = '-1'
    MERGE (r:Review {id: $review_id})
    SET r.text = $text,
        r.reviewer = $reviewer,
        r.score = $score,
        r.sentiment = $sentiment,
        r.sentiment_score = $sentiment_score
    MERGE (m)-[:HAS_REVIEW]->(r)
    """
    tx.run(query, movie_id=movie_id, review_id=review_id, text=text,
           reviewer=reviewer, score=score, sentiment=sentiment,
           sentiment_score=sentiment_score)




## Connecting Nodes

In [5]:
# Connecting Nodes

def connect_movie_genres(tx, imdb_id, genres):
    query = """
    MATCH (m:Movie {imdb_id: $imdb_id})
    WITH m, $genres AS genre_list
    UNWIND genre_list AS genre_name
    MERGE (g:Genre {name: genre_name})
    MERGE (m)-[:HAS_GENRE]->(g)
    """
    tx.run(query, imdb_id=imdb_id, genres=genres)

def connect_movie_award(tx, imdb_id, title, award_name, rel_type="WON"):
    query = f"""
    MERGE (m:Movie {{imdb_id: $imdb_id}})
      ON CREATE SET m.title = $title
    MATCH (a:Award {{name: $award_name}})
    MERGE (m)-[:{rel_type}]->(a)
    """
    tx.run(query, imdb_id=imdb_id, title=title, award_name=award_name)
    
def connect_movie_company(tx, imdb_id, company_name):
    query = """
    MATCH (m:Movie {imdb_id: $imdb_id})
    MERGE (c:Company {name: $company_name})
    MERGE (m)-[:PRODUCED_BY]->(c)
    """
    tx.run(query, imdb_id=imdb_id, company_name=company_name)
    
def connect_movie_person(tx, imdb_id, person_name, rel_type, role=None):
    query = f"""
    MATCH (m:Movie {{imdb_id: $imdb_id}})
    MERGE (p:Person {{name: $person_name}})
    SET p.roles = CASE 
        WHEN p.roles IS NULL THEN [$role]
        WHEN NOT $role IN p.roles THEN p.roles + $role
        ELSE p.roles
    END
    MERGE (m)-[:{rel_type}]->(p)
    """
    tx.run(query, imdb_id=imdb_id, person_name=person_name, role=role)





### Parsing

In [52]:
# parse for movies metadata

import re
import json
import pandas as pd

def _clean_text(s):
    if not isinstance(s, str):
        return s
    s = s.strip()
    # remove leading/trailing punctuation often found in your raws (commas, quotes, colons)
    s = re.sub(r'^[,:\s"\']+', '', s)
    s = re.sub(r'[,:\s"\']+$', '', s)
    return s.strip()

def parse_akas_to_dict(akas_str):
    if akas_str is None or (isinstance(akas_str, float) and pd.isna(akas_str)):
        return None
    s = str(akas_str).strip()
    if not s:
        return None

    pattern = r"(.+?)\s*\(([^)]+)\)"
    matches = re.findall(pattern, s)

    if not matches:
        parts = [p.strip() for p in s.split(",") if p.strip()]
        if not parts:
            return None
        return {f"title_{i+1}": _clean_text(p) for i, p in enumerate(parts)}

    aka_dict = {}
    for title, country in matches:
        title_clean = _clean_text(title)
        country_clean = _clean_text(country)
        if not country_clean:
            key = f"unknown_{len(aka_dict)+1}"
        else:
            key = country_clean
        aka_dict[key] = title_clean

    return aka_dict if aka_dict else None

def parse_age_restrictions_to_dict(age_str):
    if age_str is None or (isinstance(age_str, float) and pd.isna(age_str)):
        return None
    s = str(age_str).strip()
    if not s:
        return None

    entries = [e.strip() for e in s.split(",") if e.strip()]
    restrictions = {}
    for entry in entries:
        m = re.match(r"([^:]+):([^:]+?)(?:::\((.+?)\))?$", entry)
        if m:
            country = _clean_text(m.group(1))
            rating = _clean_text(m.group(2))
            region = m.group(3)
            if region:
                region_clean = _clean_text(region)
                key = f"{country} ({region_clean})" if country else region_clean
            else:
                key = country
            if key and rating:
                restrictions[key] = rating
        else:
            continue

    return restrictions if restrictions else None


def parse_taglines(tagline_str): # only the commas followed by capital letter is considered a new element in the list.
    if tagline_str is None or (isinstance(tagline_str, float) and pd.isna(tagline_str)):
        return None
    s = str(tagline_str).strip()
    if not s:
        return None
    # split on comma + space only when next char is uppercase
    parts = re.split(r', (?=[A-Z])', s)
    taglines = [t.strip() for t in parts if t.strip()]
    return taglines if taglines else None

def parse_keywords(keyword_str):
    if not isinstance(keyword_str, str) or not keyword_str.strip():
        return None
    
    keywords = [k.strip() for k in keyword_str.split(",") if k.strip()]
    return keywords if keywords else None

def parse_countries(country_str):
    if not isinstance(country_str, str) or not country_str.strip():
        return None
    
    countries = [c.strip() for c in country_str.split(",") if c.strip()]
    return countries if countries else None

# used in mapping the movies and genres
def parse_genres(genre_str):
    if not isinstance(genre_str, str) or not genre_str.strip():
        return []
    return [g.strip() for g in genre_str.split(",") if g.strip()]

def parse_companies(companies_str):
    if not isinstance(companies_str, str) or not companies_str.strip():
        return []
    return [c.strip() for c in companies_str.split(",") if c.strip()]

def parse_people(people_str):
    if not isinstance(people_str, str) or not people_str.strip():
        return []
    return [p.strip() for p in people_str.split(",") if p.strip()]



### Making Movie Nodes

In [38]:
# Create Movie Nodes

import pandas as pd
from tqdm import tqdm

csv_path = r"C:\Users\anura\OneDrive\Desktop\NPN Hackathon\movie_metadata\movie_meta_data.csv"
movies_df = pd.read_csv(csv_path)
with driver.session(database=database) as session:
    printed = 0
    for _, row in tqdm(movies_df.iterrows(), total=len(movies_df)):
        try:
            imdb_raw = row.get("imdbid")
            if pd.isna(imdb_raw):
                continue
            imdb_id = str(imdb_raw).strip()
            if not imdb_id:
                continue

            # basic fields
            title = str(row.get("title")).strip() if pd.notna(row.get("title")) else None
            year = row.get("year", None)
            # normalize numeric columns
            def to_int(v):
                if v is None or (isinstance(v, float) and pd.isna(v)):
                    return None
                try:
                    return int(v)
                except Exception:
                    try:
                        return int(float(v))
                    except Exception:
                        return None
            def to_float(v):
                if v is None or (isinstance(v, float) and pd.isna(v)):
                    return None
                try:
                    return float(v)
                except Exception:
                    return None

            metascore = to_int(row.get("metascore", None))
            imdb_rating = to_float(row.get("imdb user rating", None))
            imdb_votes = to_int(row.get("number of imdb user votes", None))
            budget = to_int(row.get("budget", None))

            # parse and serialize dicts -> JSON strings (safe primitive)
            akas_dict = parse_akas_to_dict(row.get("akas", None))
            akas_json = json.dumps(akas_dict, ensure_ascii=False) if akas_dict else None

            age_dict = parse_age_restrictions_to_dict(row.get("age restrict", None))
            age_json = json.dumps(age_dict, ensure_ascii=False) if age_dict else None

            # lists
            countries = parse_countries(row.get("countries", None))
            keywords = parse_keywords(row.get("keywords", None))
            taglines = parse_taglines(row.get("taglines", None))
            plot = row.get("plot", None) if pd.notna(row.get("plot", None)) else None

            params = {
                "imdb_id": imdb_id,
                "title": title,
                "year": year,
                "akas": akas_json,
                "metascore": metascore,
                "imdb_rating": imdb_rating,
                "imdb_votes": imdb_votes,
                "budget": budget,
                "countries": countries,
                "age_restrictions": age_json,
                "keywords": keywords,
                "taglines": taglines,
                "plot": plot
            }

            session.execute_write(create_movie, **params)

        except Exception as e:
            print(f"Error preparing {row.get('imdbid')}: {e}")

100%|██████████| 2858/2858 [00:38<00:00, 74.09it/s]


### Making and connecting Genre Nodes

In [39]:
# Make genre nodes

all_genres = set()

for genres in movies_df["genres"].dropna():
    for g in str(genres).split(","):
        g = g.strip()
        if g:
            all_genres.add(g)

print("Total unique genres:", len(all_genres))
print("First 10:", list(all_genres)[:10])

with driver.session(database=database) as session:
    for genre in all_genres:
        session.execute_write(create_genre, genre)

Total unique genres: 27
First 10: ['Action', 'Romance', 'Musical', 'Thriller', 'Drama', 'Animation', 'Talk-Show', 'Music', 'Game-Show', 'Film-Noir']


In [40]:
# Connecting Movie and Genre Nodes

with driver.session(database=database) as session:
    for _, row in tqdm(movies_df.iterrows(), total=len(movies_df)):
        imdb_raw = row.get("imdbid")
        if pd.isna(imdb_raw):
            continue
        imdb_id = str(imdb_raw).strip()
        if not imdb_id:
            continue

        genres = parse_genres(row.get("genres", None))
        if genres:
            try:
                session.execute_write(connect_movie_genres, imdb_id, genres)
            except Exception as e:
                print(f"Error connecting genres for {imdb_id}: {e}")

100%|██████████| 2858/2858 [00:23<00:00, 121.23it/s]


### Making and connecting Award Nodes

In [41]:
# Create Movie Award Nodes
csv_path = r"C:\Users\anura\OneDrive\Desktop\NPN Hackathon\movie_metadata\screenplay_awards.csv"
awards_df = pd.read_csv(csv_path)
award_columns = awards_df.columns[1:]
print(award_columns)

with driver.session(database=database) as session:
    for award_name in award_columns:
        session.execute_write(create_award, award_name)


Index(['Academy Awards adapted screenplay',
       'Academy Awards original screenplay', 'BAFTA nominations',
       'Golden Globe Award for Best Screenplay',
       'Writers Guild Awards Winners & Nominees 2020-2013'],
      dtype='object')


In [ ]:
import pandas as pd
from tqdm import tqdm

ALLOWED_REL_TYPES = {"WON", "NOMINATED_FOR"}

def connect_movie_award(tx, imdb_id, title, award_name, rel_type="WON"):
    if rel_type not in ALLOWED_REL_TYPES:
        raise ValueError(f"Invalid rel_type: {rel_type}")

    # Use MERGE for both nodes and MERGE the relationship.
    # Note: rel_type is inserted into the query string after validation.
    query = f"""
    MERGE (m:Movie {{imdb_id: $imdb_id}})
      ON CREATE SET m.title = $title
    MERGE (a:Award  {{name: $award_name}})
    MERGE (m)-[:{rel_type}]->(a)
    """
    tx.run(query, imdb_id=imdb_id, title=title, award_name=award_name)

csv_path = r"C:\Users\anura\OneDrive\Desktop\NPN Hackathon\movie_metadata\screenplay_awards.csv"
awards_df = pd.read_csv(csv_path)

movie_col = awards_df.columns[0]
award_columns = list(awards_df.columns[1:])

nomination_awards = {
    "BAFTA nominations",
    "Writers Guild Awards Winners & Nominees 2020-2013"
}

with driver.session(database=database) as session:
    for _, row in tqdm(awards_df.iterrows(), total=len(awards_df)):
        movie_str = row[movie_col]
        if not isinstance(movie_str, str) or "_" not in movie_str:
            continue

        try:
            title, imdb_id = movie_str.rsplit("_", 1)
            title = title.strip()
            imdb_id = imdb_id.strip()
        except Exception:
            continue

        for award_name in award_columns:
            value = row[award_name]
            if pd.isna(value) or str(value).strip() == "":
                continue  # no award info

            rel_type = "NOMINATED_FOR" if award_name in nomination_awards else "WON"

            try:
                session.execute_write(connect_movie_award, imdb_id, title, award_name, rel_type)
            except Exception as e:
                print(f"Error linking {title} ({imdb_id}) to {award_name}: {e}")


100%|██████████| 462/462 [00:21<00:00, 21.93it/s]


### Connecting Production companies

In [48]:
csv_path = r"C:\Users\anura\OneDrive\Desktop\NPN Hackathon\movie_metadata\movie_meta_data.csv"
movies_df = pd.read_csv(csv_path)

with driver.session(database=database) as session:
    for _, row in tqdm(movies_df.iterrows(), total=len(movies_df)):
        imdb_raw = row.get("imdbid")
        if pd.isna(imdb_raw):
            continue
        imdb_id = str(imdb_raw).strip()
        if not imdb_id:
            continue

        companies = parse_companies(row.get("production companies", None))
        for company in companies:
            try:
                session.execute_write(connect_movie_company, imdb_id, company)
            except Exception as e:
                print(f"Error linking {imdb_id} to {company}: {e}")


100%|██████████| 2858/2858 [01:55<00:00, 24.67it/s]


### Making and connecting Person Nodes

In [55]:
csv_path = r"C:\Users\anura\OneDrive\Desktop\NPN Hackathon\movie_metadata\movie_meta_data.csv"
movies_df = pd.read_csv(csv_path)

role_mapping = {
    "producers": "PRODUCED_BY",
    "writers": "WRITTEN_BY",
    "directors": "DIRECTED_BY",
    "casting directors": "CASTING_DIRECTED_BY",
    "cast": "ACTED_IN"
}

with driver.session(database=database) as session:
    for _, row in tqdm(movies_df.iterrows(), total=len(movies_df)):
        imdb_raw = row.get("imdbid")
        if pd.isna(imdb_raw):
            continue
        imdb_id = str(imdb_raw).strip()
        if not imdb_id:
            continue

        for col, rel in role_mapping.items():
            people = parse_people(row.get(col, None))
            for person in people:
                try:
                    session.execute_write(
                        connect_movie_person,
                        imdb_id,
                        person,
                        rel,
                        role=col[:-1] if col.endswith("s") else col
                    )
                except Exception as e:
                    print(f"Error linking {person} in {col} for {imdb_id}: {e}")



100%|██████████| 2858/2858 [2:33:26<00:00,  3.22s/it]  


In [ ]:
// Fix "cas" role to "cast" or "actor"
MATCH (p:Person)
WHERE p.role = 'cas'
SET p.role = 'actor';


MATCH (p:Person)
WHERE p.role = 'casting director'
SET p.role = 'Casting Director';

MATCH (p:Person)
WHERE p.role = 'producer'
SET p.role = 'Producer';

MATCH (p:Person)
WHERE p.role = 'writer'
SET p.role = 'Writer';

MATCH (p:Person)
WHERE p.role = 'director'
SET p.role = 'Director';


### Making and Connecting Review Nodes

In [6]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import torch

csv_path = r"C:\Users\anura\OneDrive\Desktop\NPN Hackathon\movie_metadata\metacritic_reviews_cut_versions.csv"
review_df = pd.read_csv(csv_path)


device = 0 if torch.cuda.is_available() else -1
print("Using device:", "CUDA" if device == 0 else "CPU")

sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    device=device
)

with driver.session(database=database) as session:
    for idx, row in tqdm(review_df.iterrows(), total=len(review_df)):
        review_text = str(row["text"])
        score = row.get("score", None)
        imdb_id = str(row["imdbid"])

        result = sentiment_analyzer(review_text, truncation=True, max_length=512)[0]
        sentiment = result["label"].lower()
        sentiment_score = float(result["score"])

        review_id = f"{imdb_id}_review_{idx}"

        session.execute_write(
            create_review,
            imdb_id,
            review_id,
            review_text,
            "Metacritic",
            score,
            sentiment,
            sentiment_score
        )


Using device: CUDA


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 21024/21024 [25:44<00:00, 13.61it/s] 


In [15]:
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_groq import ChatGroq
from langchain.tools import Tool
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()


def clean_response(output: str) -> str:
    import re
    return re.sub(r"<think>.*?</think>", "", output, flags=re.DOTALL).strip()

llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0.0,
    max_tokens=None,
    max_retries=2,
    timeout=None,
)

cypher_prompt = PromptTemplate(
    input_variables=["question", "schema"],
    template="""
You are a Cypher expert. Given a natural language question and the database schema, 
generate a safe and correct Cypher query.

Schema:
{schema}

Question:
{question}

Return only the Cypher query, nothing else.
""",
)

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="12345678",
    database="moviesdb"
)

graph_qa = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    cypher_prompt=cypher_prompt,
    response_format="text",
    allow_dangerous_requests=True
)

# --- Wrap in Tool
def neo4j_tool_fn(question: str) -> str:
    try:
        raw = graph_qa.run(question)
        return clean_response(raw)
    except Exception as e:
        return f"[Neo4j tool error] {type(e).__name__}: {e}"

neo4j_tool = Tool(
    name="Neo4jGraphQA",
    func=neo4j_tool_fn,
    description="Answer questions about movies/people/awards using the Neo4j graph. Generate correct Cypher queries based on schema."
)

if __name__ == "__main__":
    q = "Suggest me any 3 movies."
    print("Q:", q)
    print("A:", neo4j_tool_fn(q))


Q: Suggest me any 3 movies.
A: Here are three movie suggestions for you: A Night at the Roxbury (1998), At First Sight (1999), and The Avengers (1998).


In [ ]:
from langchain.tools import Tool

neo4j_tool = Tool(
    name="Neo4jGraphQA",
    func=graph_qa.run,
    description="Answer questions about movies, genres, and awards using the Neo4j graph database."
)
